# Code

In [1]:
# is cuda available?
import torch
"cuda" if torch.cuda.is_available() else "cpu"

'cpu'

### Create dataset loaders

In [3]:
import numpy as np
def create_balanced_split(dataset, val_samples_per_class=None):
    """
    Split dataset into train and validation sets with equal samples per class.
    If val_samples_per_class is None, uses 10% of the smallest class.
    """
    # Get all targets (class labels)
    targets = np.array(dataset.targets)
    
    # Find the size of each class
    class_sizes = []
    for class_idx in range(len(dataset.classes)):
        class_indices = np.where(targets == class_idx)[0]
        class_sizes.append(len(class_indices))
    
    # If not specified, take 10% of the smallest class
    if val_samples_per_class is None:
        val_samples_per_class = int(min(class_sizes) * 0.1)
    
    print(f"\nTaking {val_samples_per_class} samples per class for validation")
    
    train_indices = []
    val_indices = []
    
    # For each class, take equal number of samples
    for class_idx in range(len(dataset.classes)):
        # Get all indices for this class
        class_indices = np.where(targets == class_idx)[0]
        
        # Shuffle indices for this class
        np.random.shuffle(class_indices)
        
        # Take fixed number for validation
        val_indices.extend(class_indices[:val_samples_per_class])
        train_indices.extend(class_indices[val_samples_per_class:])
    
    return train_indices, val_indices

In [4]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import ImageFolder
import numpy as np
# Define data transformations for grayscale images
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.Resize((48, 48)),  # Resize to 48x48
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize for 1 channel
])

# Load the full dataset from the folder structure
full_dataset = ImageFolder(root='train_preprocessed_augmented', transform=transform)

# Set random seed for reproducibility
np.random.seed(42)

train_indices, val_indices = create_balanced_split(full_dataset, 300)

train_dataset = Subset(full_dataset, train_indices)
val_dataset = Subset(full_dataset, val_indices)

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=4
)

valid_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=4
)

# Print dataset information
print(f"Number of classes: {len(full_dataset.classes)}")
print(f"Classes: {full_dataset.classes}")
print(f"Total images: {len(full_dataset)}")
print(f"Training images: {len(train_dataset)}")
print(f"Validation images: {len(val_dataset)}")
print(f"Class to index mapping: {full_dataset.class_to_idx}")

print("\nClass distribution in training set:")
train_targets = [full_dataset.targets[i] for i in train_indices]
for class_idx, class_name in enumerate(full_dataset.classes):
    count = train_targets.count(class_idx)
    print(f"  {class_name}: {count} images")

# Print class distribution in validation set
print("\nClass distribution in validation set:")
val_targets = [full_dataset.targets[i] for i in val_indices]
for class_idx, class_name in enumerate(full_dataset.classes):
    count = val_targets.count(class_idx)
    print(f"  {class_name}: {count} images")


Taking 300 samples per class for validation
Number of classes: 6
Classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad']
Total images: 121605
Training images: 119805
Validation images: 1800
Class to index mapping: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5}

Class distribution in training set:
  angry: 19675 images
  disgust: 1880 images
  fear: 20185 images
  happy: 35775 images
  neutral: 24525 images
  sad: 17765 images

Class distribution in validation set:
  angry: 300 images
  disgust: 300 images
  fear: 300 images
  happy: 300 images
  neutral: 300 images
  sad: 300 images


### EmotionCNN

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EmotionCNN(nn.Module):
    def __init__(self, num_classes=7):
        super(EmotionCNN, self).__init__()
        
        # Block 1
        self.conv1_1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1_1 = nn.BatchNorm2d(32)
        self.conv1_2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn1_2 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.25)
        
        # Block 2
        self.conv2_1 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2_1 = nn.BatchNorm2d(64)
        self.conv2_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn2_2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout2 = nn.Dropout(0.25)
        
        # Block 3
        self.conv3_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3_1 = nn.BatchNorm2d(128)
        self.conv3_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn3_2 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout3 = nn.Dropout(0.25)
        
        # Fully connected layers
        # After 3 pooling layers: 48 -> 24 -> 12 -> 6
        self.fc1 = nn.Linear(128 * 6 * 6, 256)
        self.bn_fc = nn.BatchNorm1d(256)
        self.dropout_fc = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, num_classes)
        
    def forward(self, x):
        # Block 1
        x = F.relu(self.bn1_1(self.conv1_1(x)))
        x = F.relu(self.bn1_2(self.conv1_2(x)))
        x = self.pool1(x)
        x = self.dropout1(x)
        
        # Block 2
        x = F.relu(self.bn2_1(self.conv2_1(x)))
        x = F.relu(self.bn2_2(self.conv2_2(x)))
        x = self.pool2(x)
        x = self.dropout2(x)
        
        # Block 3
        x = F.relu(self.bn3_1(self.conv3_1(x)))
        x = F.relu(self.bn3_2(self.conv3_2(x)))
        x = self.pool3(x)
        x = self.dropout3(x)
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = F.relu(self.bn_fc(self.fc1(x)))
        x = self.dropout_fc(x)
        x = self.fc2(x)
        
        return x

### Metrics

In [1]:
import matplotlib.pyplot as plt
def metrics(num_epochs, train_losses, valid_losses, train_accuracies, valid_accuracies):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss')
    plt.plot(range(1, num_epochs + 1), valid_losses, label='Valid Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Loss overtime')

    plt.subplot(1, 2, 2)
    plt.plot(range(1, num_epochs + 1), train_accuracies, label='Train Accuracy')
    plt.plot(range(1, num_epochs + 1), valid_accuracies, label='Valid Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.title('Accuracy overtime')
    plt.savefig("metrics.png", bbox_inches='tight')
    plt.show()

### Class weights

In [6]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

def calculate_class_weights(full_dataset, train_indices):
    """
    Calculate class weights based on training set distribution
    
    Args:
        full_dataset: Full dataset with targets
        train_indices: Indices used for training
    
    Returns:
        torch.Tensor: Class weights
    """
    # Get training labels
    train_targets = [full_dataset.targets[i] for i in train_indices]
    train_targets = np.array(train_targets)
    
    # Calculate class weights using sklearn
    classes = np.unique(train_targets)
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=classes,
        y=train_targets
    )
    
    # Convert to torch tensor
    class_weights = torch.FloatTensor(class_weights)
    
    # Print class distribution and weights
    print("\nClass distribution and weights:")
    print("-" * 60)
    for class_idx, class_name in enumerate(full_dataset.classes):
        count = np.sum(train_targets == class_idx)
        weight = class_weights[class_idx].item()
        print(f"{class_name:10s}: {count:6d} images, weight: {weight:.4f}")
    print("-" * 60)
    
    return class_weights

### Training

In [ ]:

# Training loop
def train_model(model, train_loader, valid_loader, criterion, optimizer, model_name="emotionCNN", num_epochs=10):
    print("start training using ")
    print(device)
#     backbone_params = [p for name, p in model.named_parameters() if 'fc' not in name]
#     classifier_params = [p for name, p in model.named_parameters() if 'fc' in name]

#     optimizer = torch.optim.Adam([
#         {'params': backbone_params, 'lr': learning_rate * 0.1},  # Lower for pretrained
#         {'params': classifier_params, 'lr': learning_rate}       # Higher for new layer
# ])
    train_losses, valid_losses = [], []
    best_valid_loss = 1000
    train_accuracies, valid_accuracies = [], []
    
    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        # device = next(model.parameters()).device
        for i, (images, labels) in enumerate(train_loader):
            # print(images.dtype, device)
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            # if (i + 1) % 10 == 0:
            #     print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], '
            #           f'Loss: {loss.item():.4f}')
        
        train_loss = running_loss / len(train_loader)
        train_accuracy = 100 * correct / total
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        # Évaluation sur valid_loader
        model.eval()
        running_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                running_loss += loss.item()
                _, predicted = outputs.max(1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        valid_loss = running_loss / len(valid_loader)
        valid_accuracy = 100 * correct / total
        valid_losses.append(valid_loss)
        valid_accuracies.append(valid_accuracy)
 
        print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_accuracy:.2f}%")
        if best_valid_loss > valid_loss:
            torch.save(model.state_dict(), f"emotion_{model_name}_best.pth")
            print(f"Saved best new validalition loss with a value of {valid_loss}")
            best_valid_loss = valid_loss
            
    print('Finished Training')
    torch.save(model.state_dict(), f'emotion_{model_name}_final.pth')
    print(f'Model saved as emotion_{model_name}.pth')
    metrics(num_epochs, train_losses, valid_losses, train_accuracies, valid_accuracies)

### Reset training

In [ ]:

# This cell has been removed - ResNet training code

### EmotionCNN training

In [ ]:
# Create model architecture
model = EmotionCNN(num_classes=7)

# Load the saved weights
model.load_state_dict(torch.load('emotion_emotionCNN_continued_final.pth'))

# Move to device
model = model.to(device)

learning_rate = 0.0001
class_weights = calculate_class_weights(full_dataset, train_indices)
class_weights = class_weights.to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

# Continue training for 10 more epochs
train_model(model, train_loader, valid_loader, criterion, optimizer, "emotionCNN_continued", 10)

### Reset evaluation

In [ ]:
# This cell has been removed - ResNet evaluation code

### EmotionCNN evaluation

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import pandas as pd
import os
from pathlib import Path

# Emotion labels (must match your training order)
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Define the same transforms used during training
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

def predict_single_image(model, image_path, device):
    """
    Predict emotion for a single image
    
    Args:
        model: Trained EmotionCNN model
        image_path: Path to the image
        device: torch device
    
    Returns:
        str: Predicted emotion label
    """
    try:
        # Load and preprocess image
        image = Image.open(image_path).convert('L')  # Convert to grayscale
        image_tensor = transform(image).unsqueeze(0)  # Add batch dimension
        image_tensor = image_tensor.to(device)
        
        # Make prediction
        model.eval()
        with torch.no_grad():
            output = model(image_tensor)
            probabilities = F.softmax(output, dim=1)
            predicted_class = output.argmax(dim=1).item()
        
        return emotion_labels[predicted_class]
    
    except Exception as e:
        print(f"Error processing {image_path}: {str(e)}")
        return None

def make_predictions_csv(model, test_folder, csv_template, output_csv, device):
    """
    Make predictions for all images in CSV template and save results
    
    Args:
        model: Trained EmotionCNN model
        test_folder: Folder containing test images
        csv_template: Path to template CSV file
        output_csv: Path to save predictions
        device: torch device
    """
    # Read template CSV
    df = pd.read_csv(csv_template)
    
    print(f"Making predictions for {len(df)} images...")
    print("=" * 60)
    
    predictions = []
    
    for idx, row in df.iterrows():
        image_filename = row['id']
        image_path = os.path.join(test_folder, image_filename)
        
        # Check if image exists
        if not os.path.exists(image_path):
            print(f"Warning: Image not found: {image_path}")
            predictions.append('neutral')  # Default prediction
            continue
        
        # Make prediction
        emotion = predict_single_image(model, image_path, device)
        
        if emotion is None:
            emotion = 'neutral'  # Default if prediction fails
        
        predictions.append(emotion)
        
        # Print progress every 100 images
        if (idx + 1) % 100 == 0:
            print(f"Processed {idx + 1}/{len(df)} images...")
    
    # Add predictions to dataframe
    df['emotion'] = predictions
    
    # Save to CSV
    df.to_csv(output_csv, index=False)
    
    print("=" * 60)
    print(f"✓ Predictions saved to: {output_csv}")
    print(f"✓ Total predictions: {len(predictions)}")
    
    # Print prediction distribution
    print("\nPrediction distribution:")
    emotion_counts = df['emotion'].value_counts()
    for emotion, count in emotion_counts.items():
        print(f"  {emotion:10s}: {count:4d} ({count/len(df)*100:.1f}%)")



In [ ]:
# ============================================================================
# USAGE
# ============================================================================


    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = EmotionCNN(num_classes=7)
model.load_state_dict(torch.load('emotion_emotionCNN_continued_final.pth'))
model = model.to(device)
model.eval()

print(f"Model loaded on: {device}")

# Make predictions
make_predictions_csv(
    model=model,
    test_folder='test',  # Folder containing test images
    csv_template='test_template.csv',
    output_csv='predictions.csv',
    device=device
)

print("\n✓ Done! Submit 'predictions.csv' to the competition.")